In [1]:
import os, sys, subprocess
from pathlib import Path

# --- Environment Detection and Path Setup ---

# This variable will hold the absolute path to your project's root directory.
PROJECT_ROOT = None

try:
    # This block will succeed if you are in a Google Colab environment.
    from google.colab import drive
    print("✅ Running in Google Colab environment.")

    # Mount Google Drive. You will be prompted for authorization.
    drive.mount('/content/drive', force_remount=True)

    # !!! IMPORTANT !!!
    # Change this path to the location of your project folder in your Google Drive.
    colab_project_path = '/content/drive/Shareddrives/ECAIR/AI Innovation (Projects)/PAARAL/project_paaral'

    PROJECT_ROOT = Path(colab_project_path)

    # Change working directory to colab_project_path
    os.chdir(colab_project_path)

    # pip install requirements.txt, if any. NOTE: remove versions for Colab
    requirements_file = PROJECT_ROOT / 'requirements_colab.txt'
    if os.path.exists(requirements_file):
      print("\nDetected requirements.txt")
      subprocess.run([sys.executable, '-m', 'pip', 'install', '-r', str(requirements_file), '-q'], check=True)
      print("✅ Finished installing requirements of project.")

except ImportError:
    # This block will run if you are in a local Jupyter environment.
    print("✅ Running in local Jupyter environment.")

    # We assume your notebook is in the 'notebooks' directory.
    # This line gets the current working directory (e.g., /path/to/project/notebooks)
    # and goes one level up to get the project root.
    PROJECT_ROOT = Path.cwd().parent

# --- Add Project Root to Python's Path ---

# This is a crucial step. It tells Python where to look for your custom modules,
# like the 'config' module you created.
if PROJECT_ROOT and str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))
    print(f"✅ Project root added to system path: {PROJECT_ROOT}")

# --- Import and Use Your Configuration ---

# Now that the path is set up, you can import your Config class.
try:
    from config.config import Config

    # Initialize your configuration.
    # The Config class will now correctly find its own location and the config.json file.
    cfg = Config()

    print("\n🎉 Configuration loaded successfully!")
    print(f"Public data directory: {cfg.get_path('public_data')}")
    print(f"Processed data directory: {cfg.get_path('processed_data')}")

except ImportError:
    print("\n❌ Error: Could not import the 'Config' class.")
    print("Please ensure the PROJECT_ROOT path is correct and the 'config/config.py' file exists.")
except Exception as e:
    print(f"\n❌ An error occurred while loading the configuration: {e}")

✅ Running in Google Colab environment.
Mounted at /content/drive

Detected requirements.txt
✅ Finished installing requirements of project.
✅ Project root added to system path: /content/drive/Shareddrives/ECAIR/AI Innovation (Projects)/PAARAL/project_paaral

🎉 Configuration loaded successfully!
Public data directory: /content/drive/Shareddrives/ECAIR/AI Innovation (Projects)/PAARAL/project_paaral/data/public
Processed data directory: /content/drive/Shareddrives/ECAIR/AI Innovation (Projects)/PAARAL/project_paaral/data/processed


# 0. Import libraries

In [2]:
import re
import copy
import math
import time
import pickle
import multiprocessing
import numpy as np
import pandas as pd
import geopandas as gpd
from collections import Counter
import matplotlib.pyplot as plt
import igraph as iG

from config.config import Config
cf = Config()

import importlib
def reload_module(module):
    importlib.reload(module)

In [3]:
# Get the number of available CPU cores
num_cores = multiprocessing.cpu_count()

print(f"This Colab session has {num_cores} CPU cores available.")

This Colab session has 8 CPU cores available.


# 1. Build Knowledge Base
* * *
**Notes**:
* The scope of the data we will use is mostly `SY 2023-2024` to match the available data private schools have.
* To build our network, we use the following datasets from the Department of Education (DepEd) for public and private in order of importance:
    * Longitude and Latitude (as of SY 23-24)
    * SY 23-24 Enrollment
    * SY 23-24 Furnitures, namely Seats
    * SY 23-24 Public Shifting Schedule
    * SY 23-24 SHS School Offerings
    * SY 23-24 ESC Slots
    * SY 24-25 ESC & SHS VP (GASTPE) delivering Private Schools
    * SY 24-25 GASTPE Top-ups
    * SY 24-25 ESC Tagged Learners in LIS

In [9]:
from modules import datasets

In [10]:
s_time = time.time()

kb = datasets.KnowledgeBase(load_knowledge_base=True)
e_time = time.time() - s_time

print(f"Time elapsed for building knowledge base: {(e_time / 60):.2f} minutes")

Loading public school coordinates as of SY 2023-2024.
Time elapsed for public school coordinates: 5.58 seconds

Loading private school coordinates as of 2024.
Time elapsed for private school coordinates: 4.41 seconds

Loading public and private school enrollment & SHS offerings for SY 2023-2024.
Time elapsed for enrollment & SHS offerings: 9.78 seconds

Loading public & private school furnitures, namely seats, for SY 2023-2024.
Time elapsed for public & private seats: 12.10 seconds

Loading public school shifting schedule for SY 2023-2024.
Time elapsed for public shifting: 45.10 seconds

Loading private school ESC and SHS VP delivering schools as of 2024.
Time elapsed for public shifting: 1.36 seconds

Time elapsed for building knowledge base: 1.31 minutes


# 2. Prepare Map Resources
* * *
We mainly use the `osmnx` Python library to query and locally save the graph networks of roads (and walkable paths) that will be used to build our network of schools.

Based on experience, only specialized and high-power machines are able to query the graph network of the entire Philippines in one call. Given the limitations of the machine used in this project, we broke our `osmnx` graph network queries by region and stored them locally in the directory `data/networks/regional_drive_graphs`. The relevant regional graph networks for this research are Region III, Region IV-A, Region V, and NCR.

In [ ]:
from modules import map_resources

In [ ]:
s_time = time.time()

# Initialize class that also preloads and reorganizes PSGC shapefiles
mr = map_resources.MapResources(preloaded=False)

# The gpd_gadm is the dataframe of the shapefiles of Philippines following PSGC with EPSG:4326
gpd_gadm = mr.gadm.copy()

# The output is a dictionary of complete filepaths where the keys are the PSGC region names
# of our 17 regions. These are OSMNX drive graph networks
mr.get_filepaths_of_regional_road_networks()

# Additional preprocessing is done on our compiled public & private datasets
public = kb.compile_public_datasets()
private = kb.compile_private_datasets()

gdf_public = mr.preprocess_public_school_coordinates_further(public)
gdf_private = mr.preprocess_private_school_coordinates_further(private)

e_time = time.time() - s_time
print(f"Time elapsed for preparing map resources: {(e_time / 60):.2f} minutes")

/usr/local/lib/python3.12/dist-packages/pyogrio/geopandas.py:275: UserWarning: More than one layer found in 'BgySubMuns': 'BgySubMuns.shp' (default), 'PH_Adm4_BgySubMuns.shp'. Specify layer parameter to avoid this warning.
  result = read_func(


Time elapsed for preparing map resources: 0.33 minutes


# 3. Graph Network Generation
* * *
We will use our `optimized_network_builder` module to facilitate the preparation and generation of the graph network of our schools using our compiled knowledge base and organized shapefiles. We provide a high-level overview below of how the project generates a graph network of schools.

<dl>
    Given the PSGC code of a target region:
    <ol type="1">
      <li>We get the shapefiles/geographies of the target region and its adjacent localities</li>
      <li>Using the above shapefiles, we extract the public schools in the target region and extract private schools that fall within the target and adjacent areas</li>
      <li>Still using the shapefiles, we cutout the drive network from our preloaded & locally saved OSMNX graph networks</li>
      <li>Feed the following to our network builder:
        <ol type="1">
          <li>Target & adjacent shapefiles</li>
          <li>Public and private schools</li>
          <li>Drive graph network</li>
        </ol>
      </li>
      <li>Run network builder algorithm</li>
    </ol>
</dl>

## 3.1. Parameter setting

We extract the shapes of our target geography and its adjacent geographies. `max_depth` here determines how far the algorithm will crawl outward from the target region when extracting adjacent geographies.

In [ ]:
# As mentioned in the paper, we use the geographic code of Region IV-A
target_psgc = '0400000000'
max_depth = 3
geo_results = mr.get_adjacent_geographies(
    gpd_gadm=gpd_gadm,
    target_psgc=target_psgc,
    max_depth=max_depth
)
print()

# Extracting schools using extracted geographies
school_results = mr.extract_schools_from_geographies(
    geography_results=geo_results,
    gdf_public=gdf_public,
    gdf_private=gdf_private
)
print()

Finding adjacent geographies for PSGC 0400000000 with max_depth=3
Detected administrative level: region
Finding adjacent areas at depth 1...
Found 62 adjacent areas at depth 1
Finding adjacent areas at depth 2...
Found 101 adjacent areas at depth 2
Finding adjacent areas at depth 3...
Found 136 adjacent areas at depth 3
Geography processing completed in 11.45 seconds

Extracting schools from geographies...
Prepared geometries in 0.00 seconds
Extracting public schools...
Public target query completed in 0.10 seconds
Found 3548 public schools in target area
Public adjacent depth 1 query completed in 0.02 seconds
Found 118 public schools in adjacent areas at depth 1
Public adjacent depth 2 query completed in 0.03 seconds
Found 174 public schools in adjacent areas at depth 2
Public adjacent depth 3 query completed in 0.03 seconds
Found 203 public schools in adjacent areas at depth 3
Extracting private schools...
Private search area query completed in 7.86 seconds
Found 2725 private schools

In [ ]:
%%time
# We cull a subgraph from locally saved regional drive graph networks
G_sub = mr.generate_subgraph(geo_results)

Generating subgraph from geo_results
Subgraph extracted. Time elapsed: 1.90 minutes
CPU times: user 1min 47s, sys: 3.17 s, total: 1min 50s
Wall time: 1min 55s


In [ ]:
# We explicitly identify the major groups that will comprise the school network
# This will be removed in the future since we will connect schools with each other regardless of groups
public_in_city = school_results['public_schools_in_target'].copy()

# My private dataframe has columns with "validated" coordinates - we explicitly call them here
private_in_space = school_results['private_schools_in_target'].copy()
private_in_space = private_in_space.drop(columns=['longitude','latitude'])
private_in_space = private_in_space.rename(
    columns={'longitude_valid':'longitude', 'latitude_valid':'latitude'}
)

# Public schools adjacent
adjacent_dfs = [v for k,v in school_results['public_schools_in_adjacent'].items()]
public_periph = pd.concat(adjacent_dfs)

# Private schools adjacent
adjacent_dfs = [v for k,v in school_results['private_schools_in_adjacent'].items()]
private_periph = pd.concat(adjacent_dfs)
private_periph = private_periph.drop(columns=['longitude','latitude'])
private_periph = private_periph.rename(
    columns={'longitude_valid':'longitude', 'latitude_valid':'latitude'}
)

# A dataframe of adjacent PUBLIC and PRIVATE schools
all_periph = pd.concat([public_periph, private_periph])

## 3.2. Graph Generation Algorithm

In [ ]:
from modules import optimized_network_builder as onb

In [ ]:
# Initialize the optimized builder
network_builder = onb.OptimizedSchoolNetworkBuilder(
    road_network_graph=G_sub,
    public_schools_gdf=public_in_city.reset_index(),
    private_schools_gdf=private_in_space.reset_index(),
    peripheral_schools_gdf=all_periph.reset_index(),
    admin_boundary=geo_results['target_area'],
    num_processes=None  # None means parallelization will use all available CPUs
)

In [ ]:
%%time
# Build the complete network in one call
results = network_builder.build_complete_network(
    buffer_distance_m=5000,
    max_distance_km=15
)
print(results.keys())

🚀 Starting optimized network build...
📊 Setting up master infrastructure...
School counts: Public=3548, Private=2003, Peripheral=1217
Combined 6768 total schools
🔄 Converting NetworkX to iGraph...
Projected graph: 458252 nodes, 1146971 edges
Created iGraph with 458252 vertices and 1146971 edges
🔄 Creating WGS84 version of master iGraph...
Created iGraph with 458252 vertices and 1146971 edges
Created 4326 iGraph with 458252 vertices
🗺️  Mapping schools to road network vertices...
Mapping results: 6768 successful, 0 failed
✅ Master graph: 458252 vertices, 1146971 edges
✅ Master graph 4326: 458252 vertices
✅ Mapped 6768 schools to road network
🌍 Building spatial indices...
✅ Built spatial indices for 3 school types

🐛 DEBUG: Testing single school...
Testing school ID: 107329
✅ School 107329 mapped to vertex 406322
Found 10 nearby schools: ['107318', '107329', '107334', '107319', '342211']...
Distance matrix shape: (1, 5)
Distance matrix:
         107318  107329    107334    107319    3422

In [ ]:
# Run #1 2:46 PM
graph = results['distance_graph']
count_nodes = len(graph.vs)
count_edges = len(graph.es)
print(f"Number of nodes: {count_nodes}")
print(f"Number of nodes: {count_edges}")

Number of nodes: 6230
Number of nodes: 407545


In [ ]:
# Run #2 2:58 PM
graph = results['distance_graph']
count_nodes = len(graph.vs)
count_edges = len(graph.es)
print(f"Number of nodes: {count_nodes}")
print(f"Number of nodes: {count_edges}")

Number of nodes: 6230
Number of nodes: 407545


In [ ]:
# Run #3 3:25 PM
graph = results['distance_graph']
count_nodes = len(graph.vs)
count_edges = len(graph.es)
print(f"Number of nodes: {count_nodes}")
print(f"Number of nodes: {count_edges}")

Number of nodes: 6230
Number of nodes: 407545


In [ ]:
# Run #4 12:35 PM
graph = results['distance_graph']
count_nodes = len(graph.vs)
count_edges = len(graph.es)
print(f"Number of nodes: {count_nodes}")
print(f"Number of nodes: {count_edges}")

Number of nodes: 6230
Number of nodes: 407545


In [ ]:
# Save Run #3
savepath = os.path.join("output","gnet_flow_Region4A_08-27-2025.pkl")
with open(savepath, 'wb') as file: # Open the file in binary write mode ('wb')
    pickle.dump(results, file) # Save the dictionary to the file

In [4]:
# load_path = os.path.join("output", "gnet_flow_Region4A_07-23-2025.pkl")
load_path = os.path.join("output","gnet_flow_Region4A_08-27-2025.pkl")
with open(load_path, 'rb') as file: # Open the file in binary read mode ('rb')
    saved_results = pickle.load(file) # Load the dictionary from the file

sgraph = saved_results['distance_graph']
count_nodes = len(sgraph.vs)
count_edges = len(sgraph.es)
print(f"Number of nodes: {count_nodes}")
print(f"Number of nodes: {count_edges}")

Number of nodes: 6230
Number of nodes: 407545


In [ ]:
# # Old saved state for AAAI
# load_path = os.path.join("output", "gnet_flow_Region4A_07-23-2025.pkl")
# # load_path = os.path.join("output","gnet_flow_Region4A_08-27-2025.pkl")
# with open(load_path, 'rb') as file: # Open the file in binary read mode ('rb')
#     saved_results = pickle.load(file) # Load the dictionary from the file

# tgraph = saved_results['distance_graph']
# count_nodes = len(tgraph.vs)
# count_edges = len(tgraph.es)
# print(f"Number of nodes: {count_nodes}")
# print(f"Number of nodes: {count_edges}")

Number of nodes: 6332
Number of nodes: 395927


## 3.3 Inspect graph

In [ ]:
# Public
g = saved_results['distance_graph']

pub_nodes = [v for v in g.vs if v['school_type'] == "public"]
display(pub_nodes[0])

print()

priv_nodes = [v for v in g.vs if v['school_type'] == "private"]
display(priv_nodes[0])

igraph.Vertex(<igraph.Graph object at 0x7c5b5c256a50>, 0, {'school_id': '101583', 'school_type': 'public', 'school_name': 'Dorongan ES', 'x': 560327.2979985803, 'y': 1600265.3025909872, 'school_attrs': [{'region': 'Region I', 'division': 'Pangasinan I, Lingayen', 'school_name': 'Dorongan ES', 'province': 'PANGASINAN', 'municipality': 'MANGATAREM', 'longitude': 121.5609412, 'latitude': 14.4687248, 'shifting_schedule': 'No Shift', 'modified coc': 'Purely ES', 'seats_es': 67, 'seats_jhs': nan, 'seats_shs': nan, 'enrollment_es': 181.0, 'enrollment_jhs': nan, 'enrollment_shs': nan, 'shs_ABM': nan, 'shs_ARTS & DESIGN': nan, 'shs_GAS': nan, 'shs_HUMSS': nan, 'shs_PBM': nan, 'shs_SPORTS': nan, 'shs_STEM': nan, 'shs_TVL': nan, 'sector': 'Public', 'x': 560327.2979985803, 'y': 1600265.3025909872}], 'is_school': True})

igraph.Vertex(<igraph.Graph object at 0x7c5b5c256a50>, 3660, {'school_id': '400395', 'school_type': 'private', 'school_name': 'Acacia School Foundation, Inc', 'x': 508749.29046636156, 'y': 1574179.26249036, 'school_attrs': [{'region': 'Region IV-A', 'division': 'Sta. Rosa City', 'school_name': 'Acacia School Foundation, Inc', 'province': 'LAGUNA', 'municipality': 'CITY OF SANTA ROSA', 'barangay': 'DON JOSE', 'school type': 'Mother school', 'modified coc': 'All Offering', 'enrollment_es': 139.0, 'enrollment_jhs': 51.0, 'enrollment_shs': 18.0, 'seats_es': 123, 'seats_jhs': 53, 'seats_kinder': 70, 'seats_shs': 18, 'esc_participating': nan, 'shsvp_participating': nan, 'esc_school_id': nan, 'esc_(tuition)': nan, 'esc_(other)': nan, 'esc_(misc)': nan, 'esc_(total)': nan, 'shsvp_(tuition)': nan, 'shsvp_(other)': nan, 'shsvp_(misc)': nan, 'shsvp_(total)': nan, 'esc_amount': nan, 'shs_ABM': 0.0, 'shs_ARTS & DESIGN': 0.0, 'shs_GAS': 1.0, 'shs_HUMSS': 0.0, 'shs_PBM': 0.0, 'shs_SPORTS': 0.0, 'shs_

In [ ]:
# Public
g = saved_results['distance_graph']

edges = [edge for edge in g.es]
display(edges[100:105])

[igraph.Edge(<igraph.Graph object at 0x7c5b5c256a50>, 100, {'length': 5073.315999999999}),
 igraph.Edge(<igraph.Graph object at 0x7c5b5c256a50>, 101, {'length': 4213.129}),
 igraph.Edge(<igraph.Graph object at 0x7c5b5c256a50>, 102, {'length': 4505.152}),
 igraph.Edge(<igraph.Graph object at 0x7c5b5c256a50>, 103, {'length': 4626.905}),
 igraph.Edge(<igraph.Graph object at 0x7c5b5c256a50>, 104, {'length': 6445.915999999999})]

## 3.4. Visualization of Graph Network

In [ ]:
# %%time
# # Sanity check if we capture our schools in Region IV-A
# fig, ax = plt.subplots(figsize=(8,8))

# mult_savefig = 1
# # Plot network anchored on the schools' lon-lat
# iG.plot(
#     results,
#     target=ax,
#     vertex_size=2*mult_savefig,
#     vertex_frame_width=.1,
#     edge_width=.1,           # Make edges thicker
#     edge_arrow_size=2*mult_savefig,      # Control arrow size
#     edge_arrow_width=2*mult_savefig,     # Control arrow width
#     bbox=(1000,1000),
#     margin=0
# )

# geo_results['target_area'].to_crs(3123).plot(
#     ax=ax, facecolor='none', edgecolor='navy',
#     linewidth=.25, alpha=1,
# )

# df_adjs = []
# for k, v in geo_results['adjacent_areas'].items():
#     df_adj = geo_results['adjacent_areas'].get(k)
#     df_adj.to_crs(3123).plot(
#         ax=ax, facecolor='lightgray', edgecolor='grey',
#         linewidth=.25, alpha=.1
#     )
#     df_adjs.append(df_adj.to_crs(3123))
# df_adjs = pd.concat(df_adjs)

# df_cm = geo_results['target_area'].to_crs(3123).copy()
# cm_3123_shp = df_cm.unary_union # This is 4326

# min_x, min_y, max_x, max_y = cm_3123_shp.bounds
# multiplier = 0.001
# ax.set_xlim(min_x*(1 - multiplier), max_x*(1 + multiplier))
# ax.set_ylim(min_y*(1 - multiplier), max_y*(1 + multiplier))

# plt.show()
# plt.close()

Buffered data was truncated after reaching the output size limit.

# 4. Experiments

## 4.1. Results of Deterministic Algorithms

In [11]:
from modules import experiments_v2 as experiments

In [12]:
graph = copy.deepcopy(sgraph)

In [13]:
geography = {"level":"region", "name":"Region IV-A"}
exp = experiments.Experiments(graph, kb, geography)

In [20]:
# For debugging
pkg = exp._initialize_experiment()

# Debug 1: Check Grade 6 enrollment data
print("=== GRADE 6 ENROLLMENT DEBUG ===")
print(f"G6 enrollment shape: {pkg['g6_enrollment'].shape}")
print(f"Total G6 students: {pkg['g6_enrollment']['enr_grade_6'].sum()}")
print(f"G6 enrollment sample:\n{pkg['g6_enrollment'].head()}")
print(f"G6 enrollment data types:\n{pkg['g6_enrollment'].dtypes}")

# Debug 2: Check school connectivity
print("\n=== SCHOOL CONNECTIVITY DEBUG ===")
public_jhs_nodes = pkg["public_jhs"]["nodes"]
es_cocs = pkg["filters"]["es_cocs"]
nearby_distance = pkg["filters"]["distance"]

total_es_connections = 0
for i, jhs_node in enumerate(public_jhs_nodes[:10]):  # Check first 10 JHS
    es_nodes = exp._get_nearby_es_nodes(jhs_node, es_cocs, nearby_distance)
    total_es_connections += len(es_nodes)
    if i < 3:  # Print details for first 3
        print(f"JHS {jhs_node['school_id']} has {len(es_nodes)} nearby ES schools")

print(f"Average ES connections per JHS: {total_es_connections/min(10, len(public_jhs_nodes))}")

# Debug 3: Check school ID types in graph vs enrollment data
print("\n=== SCHOOL ID TYPE DEBUG ===")
graph_school_ids = [v['school_id'] for v in exp.graph.vs][:5]
enrollment_school_ids = pkg['g6_enrollment'].index[:5].tolist()
print(f"Graph school ID sample: {graph_school_ids} (type: {type(graph_school_ids[0]) if graph_school_ids else 'None'})")
print(f"Enrollment school ID sample: {enrollment_school_ids} (type: {type(enrollment_school_ids[0]) if enrollment_school_ids else 'None'})")

=== GRADE 6 ENROLLMENT DEBUG ===
G6 enrollment shape: (60051, 1)
Total G6 students: 2183346.0
G6 enrollment sample:
           enr_grade_6
school_id             
100000            39.0
100001            11.0
100002            69.0
100003            16.0
100004            14.0
G6 enrollment data types:
enr_grade_6    float64
dtype: object

=== SCHOOL CONNECTIVITY DEBUG ===
JHS 300536 has 15 nearby ES schools
JHS 301067 has 0 nearby ES schools
JHS 301068 has 15 nearby ES schools
Average ES connections per JHS: 8.5

=== SCHOOL ID TYPE DEBUG ===
Graph school ID sample: ['101583', '101701', '102178', '102259', '102290'] (type: <class 'str'>)
Enrollment school ID sample: ['100000', '100001', '100002', '100003', '100004'] (type: <class 'str'>)


In [21]:
pkg = exp._initialize_experiment()
print(f"Overall congestion without incomning Grade 7 students: {pkg['current_jhs_congestion']}")
print(pkg.keys())

Overall congestion without incomning Grade 7 students: 1.5646315724992423
dict_keys(['g6_enrollment', 'distribution_graph', 'filters', 'public_jhs', 'esc_jhs', 'current_jhs_congestion', 'esc_slots'])


In [22]:
# Run Aug 27, 2025
baseline_score = exp.run_baseline_experiment(pkg)
baseline_jhs, baseline_total = exp.track_actual_jhs_enrollment(pkg['distribution_graph'])
print(f"{baseline_total:,}")
print(f"Overall congestion when ALL incoming Grade 7 students enroll in public high schools only: {baseline_score}")

272,230
Overall congestion when ALL incoming Grade 7 students enroll in public high schools only: 2.120847580583717


In [ ]:
# baseline_score = exp.run_baseline_experiment(pkg)
# baseline_jhs, baseline_total = exp.track_actual_jhs_enrollment(pkg['distribution_graph'])
# print(f"{baseline_total:,}")
# print(f"Overall congestion when ALL incoming Grade 7 students enroll in public high schools only: {baseline_score}")

297,498
Overall congestion when ALL incoming Grade 7 students enroll in public high schools only: 2.1515462410026913


In [23]:
# Run Sept. 9, 2025
pkg = exp._initialize_experiment()
redist = exp.run_redistribution_experiment(pkg)
print(redist.keys())

pkg["g6_enrollment"] = redist['redistributed_g6_enrollment']
redist_score = exp.run_baseline_experiment(pkg)
redist_jhs, redist_total = exp.track_actual_jhs_enrollment(pkg['distribution_graph'])
print(f"{redist_total:,}")
print(f"Overall congestion when a portion of incoming Grade 7 students are redirected to private schools participating in ESC: {redist_score}")

dict_keys(['redistribution_results', 'redistributed_g6_enrollment', 'involved_es_school_ids'])
227,207
Overall congestion when a portion of incoming Grade 7 students are redirected to private schools participating in ESC: 2.051252910416785


In [ ]:
# pkg = exp._initialize_experiment()
# redist = exp.run_redistribution_experiment(pkg)
# print(redist.keys())

# pkg["g6_enrollment"] = redist['redistributed_g6_enrollment']
# redist_score = exp.run_baseline_experiment(pkg)
# redist_jhs, redist_total = exp.track_actual_jhs_enrollment(pkg['distribution_graph'])
# print(f"{redist_total:,}")
# print(f"Overall congestion when a portion of incoming Grade 7 students are redirected to private schools participating in ESC: {redist_score}")

dict_keys(['redistribution_results', 'redistributed_g6_enrollment', 'involved_es_school_ids'])
254,467
Overall congestion when a portion of incoming Grade 7 students are redirected to private schools participating in ESC: 2.0842791375046574


In [24]:
looped_optimization = exp.run_looped_esc_slot_optimization(max_iterations=10)
print(looped_optimization.keys())
print(f"\nOverall congestion when the allocation of slots in ESC schools are optimized: {looped_optimization['final_congestion_score']}")


🔁 Iteration 1/10
📉 Congestion Score: 2.051253

🔁 Iteration 2/10
📉 Congestion Score: 2.051320
✅ Slot allocation converged. Stopping early.

📊 Optimization History:
  Iter 1: 2.051253
  Iter 2: 2.051320
dict_keys(['final_congestion_score', 'iterations', 'history', 'optimized_esc_slots', 'total_jhs_enrollment', 'jhs_enrollment_by_school'])

Overall congestion when the allocation of slots in ESC schools are optimized: 2.051319798022183


In [ ]:
# looped_optimization = exp.run_looped_esc_slot_optimization(max_iterations=10)
# print(looped_optimization.keys())
# print(f"\nOverall congestion when the allocation of slots in ESC schools are optimized: {looped_optimization['final_congestion_score']}")


🔁 Iteration 1/10
📉 Congestion Score: 2.084279

🔁 Iteration 2/10
📉 Congestion Score: 2.051309
✅ Slot allocation converged. Stopping early.

📊 Optimization History:
  Iter 1: 2.084279
  Iter 2: 2.051309
dict_keys(['final_congestion_score', 'iterations', 'history', 'optimized_esc_slots', 'total_jhs_enrollment', 'jhs_enrollment_by_school'])

Overall congestion when the allocation of slots in ESC schools are optimized: 2.0513085046597257


## 4.2 Finalized Results

In [25]:
index = [
    'Baseline Distribution',
    'Student Redistribution',
    'Optimized Allocation',
]
values = [
    np.round(baseline_score, 3),
    np.round(redist_score, 3),
    np.round(looped_optimization['final_congestion_score'], 3)
]
columns = ['Metric']

df_results = pd.DataFrame(
    index=index,
    data=values,
    columns=columns
)

In [ ]:
df_results

,Metric
Baseline Distribution,2.152
Student Redistribution,2.084
Optimized Allocation,2.051
